In [29]:
import operator

### Harinarayan, Rajaraman, Ullman Lattice 

In [ ]:
#The order of the dictionary entries matter because...dependency,etc
lattice={
        'abcd':100,
        'abc':50,'acd':75,
        'ab':20,'ac':30,'cd':40,
        'a':1,'c':10
        }

### Trinidad Serna Lattice

In [108]:
lattice={
    'abcde': 53799,
    'abce':47133,'abde':18456,'acde':603,'bcde':32918,
    'abe':13973, 'ace':184, 'ade':58, 'bce':26058,'bde':8186,'cde':48,
    'ae':19, 'be':5329, 'ce':12,'de':4,
    'e':1
}

### Ulises Teyechea Lattice

In [135]:
lattice={
    'abcde': 1023860,
    'abce':1019641,'abde':18456,'acde':290355,'bcde':49909,
    'abe':975300, 'ace':241820, 'ade':54765, 'bce':31171,'bde':2196,'cde':66,
    'ae':31543, 'be':1458, 'ce':34,'de':2,
    'e':1
}

In [136]:
views=list(lattice.keys())
views

['abcde',
 'abce',
 'abde',
 'acde',
 'bcde',
 'abe',
 'ace',
 'ade',
 'bce',
 'bde',
 'cde',
 'ae',
 'be',
 'ce',
 'de',
 'e']

In [137]:
selectedViews=['abcd'] #Always selected by default, also known as Top View

In [138]:
def dependencyCheck(view,views):
    #compare dependency of view1 with view2
    #if views is a single element compare individual dependency if it is a set give a count 
    #and output a list of dependent views. 
    
    dependentViewsCount=0
    dependentViews=[]
    for anotherView in views:
        n=0
        for dimension in view:
            #print('dimension',dimension)
            for subDimension in anotherView:
                #print('sub dimension',subDimension)
                if dimension == subDimension: 
                    n+=1
                    #print('found, count ', n )
        if n == len(anotherView):
            dependentViews.append(anotherView)
            #print(n,len(anotherView))
            #print('dependent view found',dependentViewsCount)
    return dependentViews

In [139]:
for view in views:
    print('La vista ',view,'tiene estas vistas dependientes ',dependencyCheck(view,views))

La vista  abcde tiene estas vistas dependientes  ['abcde', 'abce', 'abde', 'acde', 'bcde', 'abe', 'ace', 'ade', 'bce', 'bde', 'cde', 'ae', 'be', 'ce', 'de', 'e']
La vista  abce tiene estas vistas dependientes  ['abce', 'abe', 'ace', 'bce', 'ae', 'be', 'ce', 'e']
La vista  abde tiene estas vistas dependientes  ['abde', 'abe', 'ade', 'bde', 'ae', 'be', 'de', 'e']
La vista  acde tiene estas vistas dependientes  ['acde', 'ace', 'ade', 'cde', 'ae', 'ce', 'de', 'e']
La vista  bcde tiene estas vistas dependientes  ['bcde', 'bce', 'bde', 'cde', 'be', 'ce', 'de', 'e']
La vista  abe tiene estas vistas dependientes  ['abe', 'ae', 'be', 'e']
La vista  ace tiene estas vistas dependientes  ['ace', 'ae', 'ce', 'e']
La vista  ade tiene estas vistas dependientes  ['ade', 'ae', 'de', 'e']
La vista  bce tiene estas vistas dependientes  ['bce', 'be', 'ce', 'e']
La vista  bde tiene estas vistas dependientes  ['bde', 'be', 'de', 'e']
La vista  cde tiene estas vistas dependientes  ['cde', 'ce', 'de', 'e']
La

In [140]:
def additionalBenefit(views,lattice):
    benefit=[]
    minBenefit=0
    for v in views:
        if v not in selectedViews:
            #print(v)
            dependentViewsCount=dependencyCheck(selectedViews[-1],[v])
            if len(dependentViewsCount) <= 0:
                if lattice[v] < lattice[selectedViews[-1]]:
                    
                    addBenefit=abs(lattice[v]-lattice[selectedViews[-1]])
                    benefit.append((addBenefit,v))
                    
    if benefit:
        minBenefit=min(benefit)
        #print(minBenefit,v)
    return minBenefit

In [141]:
def isConnected(view1,view2):
    isParent=dependencyCheck(view1,[view2])
    if len(isParent) > 0:
        return True
    else:
        return False

In [142]:
isConnected('abc','a')

True

In [143]:
def getParentView(view,lattice,selectedViews):
    parents=[]
    for v in lattice:
        if v != view:
            if isConnected(v,view):
                parents.append(v)
    #print(parents)
    #parents will alwayas have at least the top view
    newParent=selectedViews[0] # Top View
    for parent in parents:
            if parent in selectedViews:
                if lattice[parent]<lattice[newParent]: #select as parent the one with the smallest storage cost
                    newParent=parent
                    #print('newParent ',newParent)
    return newParent  

In [144]:
#Test if c which has two selected views as parents correctly chooses the cheapest one
#selectedViews=['abcd','abc','cd']
#getParentView('c',lattice,selectedViews)

In [145]:
def getBenefit(views,lattice,selectedViews):
    B={}
    for view in lattice:
        storageCostDifference=0
        benefit=[]
        dependentViews=[]
        if view not in selectedViews:
            parent=getParentView(view,lattice,selectedViews)       
            storageCostDifference=abs(lattice[view]-lattice[parent])
            
            #Count the number of dependent Views
            view_dependentViews=dependencyCheck(view,views)
            
            
            materialized_view_dependentViews=[]
            #for selectedView in selectedViews:    
            #    parent=getParentView(view,lattice,selectedViews)
            #    if parent != selectedView and parent==selectedViews[0]:#Original
            #        materialized_view_dependentViews.extend(dependencyCheck(selectedView,views))
            
            
            for selectedView in selectedViews:
                if selectedView != parent and selectedView!=selectedViews[0]:
                    materialized_view_dependentViews.extend(dependencyCheck(selectedView,views))
                
                
                
                
            dependentViews=\
            set(view_dependentViews).difference(set(materialized_view_dependentViews))
            if dependentViews==set():
                dependentViews=set(view)
            #estimate the difference only for those selected nodes that are not parents
        
            addBenefit=additionalBenefit(views,lattice)
            if addBenefit and addBenefit[1]== view:  
                benefit.append((view,storageCostDifference,len(dependentViews),\
                dependentViews,storageCostDifference*len(dependentViews)+addBenefit[0]))
                B[view]=storageCostDifference*len(dependentViews)+addBenefit[0]
            else:
                benefit.append((view,storageCostDifference,len(dependentViews),\
                dependentViews,storageCostDifference*len(dependentViews)))
                B[view]=storageCostDifference*len(dependentViews)
                
        print(benefit)
        
        #Update Selected Views
        #B[view]=storageCostDifference*len(dependentViews)+addBenefit[0]
        
    update=max(B.items(), key=operator.itemgetter(1))[0]
    selectedViews.append(update)
        
    return selectedViews

In [146]:
selectedViews=['abcde']
getBenefit(views,lattice,selectedViews)
selectedViews

[]
[('abce', 4219, 8, {'ce', 'abce', 'ae', 'abe', 'be', 'ace', 'bce', 'e'}, 33752)]
[('abde', 1005404, 8, {'bde', 'ae', 'abe', 'abde', 'be', 'ade', 'de', 'e'}, 8043232)]
[('acde', 733505, 8, {'ce', 'cde', 'ae', 'acde', 'ade', 'ace', 'de', 'e'}, 5868040)]
[('bcde', 973951, 8, {'bde', 'ce', 'cde', 'bcde', 'be', 'bce', 'de', 'e'}, 7791608)]
[('abe', 48560, 4, {'be', 'ae', 'abe', 'e'}, 194240)]
[('ace', 782040, 4, {'ae', 'ce', 'ace', 'e'}, 3128160)]
[('ade', 969095, 4, {'ae', 'ade', 'de', 'e'}, 3876380)]
[('bce', 992689, 4, {'be', 'ce', 'bce', 'e'}, 3970756)]
[('bde', 1021664, 4, {'bde', 'be', 'de', 'e'}, 4086656)]
[('cde', 1023794, 4, {'ce', 'cde', 'de', 'e'}, 4095176)]
[('ae', 992317, 2, {'ae', 'e'}, 1984634)]
[('be', 1022402, 2, {'be', 'e'}, 2044804)]
[('ce', 1023826, 2, {'ce', 'e'}, 2047652)]
[('de', 1023858, 2, {'de', 'e'}, 2047716)]
[('e', 1023859, 1, {'e'}, 1023859)]


['abcde', 'abde']

In [148]:
getBenefit(views,lattice,selectedViews)
selectedViews

[]
[('abce', 4219, 2, {'ace', 'abce'}, 8438)]
[]
[('acde', 733505, 2, {'acde', 'ace'}, 1467010)]
[]
[('abe', 956844, 2, {'ae', 'abe'}, 1913688)]
[('ace', 782040, 1, {'ace'}, 782040)]
[('ade', 36309, 2, {'ae', 'ade'}, 72618)]
[('bce', 18738, 2, {'ce', 'bce'}, 37476)]
[('bde', 16260, 3, {'d', 'e', 'b'}, 48780)]
[('cde', 49843, 2, {'ce', 'cde'}, 99686)]
[('ae', 13087, 1, {'ae'}, 31453)]
[('be', 16998, 2, {'e', 'b'}, 33996)]
[('ce', 49875, 1, {'ce'}, 49875)]
[('de', 18454, 2, {'d', 'e'}, 36908)]
[('e', 18455, 1, {'e'}, 18455)]


['abcde', 'abde', 'bcde', 'abe']

In [149]:
getBenefit(views,lattice,selectedViews)
selectedViews

[]
[('abce', 4219, 2, {'ace', 'abce'}, 8438)]
[]
[('acde', 733505, 2, {'acde', 'ace'}, 2151955)]
[]
[]
[('ace', 782040, 1, {'ace'}, 782040)]
[('ade', 36309, 1, {'ade'}, 36309)]
[('bce', 18738, 2, {'ce', 'bce'}, 37476)]
[('bde', 16260, 3, {'d', 'e', 'b'}, 48780)]
[('cde', 49843, 2, {'ce', 'cde'}, 99686)]
[('ae', 13087, 2, {'a', 'e'}, 26174)]
[('be', 16998, 2, {'e', 'b'}, 33996)]
[('ce', 49875, 1, {'ce'}, 49875)]
[('de', 18454, 2, {'d', 'e'}, 36908)]
[('e', 18455, 1, {'e'}, 18455)]


['abcde', 'abde', 'bcde', 'abe', 'acde']

In [150]:
getBenefit(views,lattice,selectedViews)
selectedViews

[]
[('abce', 4219, 1, {'abce'}, 4219)]
[]
[]
[]
[]
[('ace', 48535, 1, {'ace'}, 48535)]
[('ade', 36309, 3, {'d', 'a', 'e'}, 108927)]
[('bce', 18738, 1, {'bce'}, 277922)]
[('bde', 16260, 3, {'d', 'e', 'b'}, 48780)]
[('cde', 49843, 3, {'d', 'c', 'e'}, 149529)]
[('ae', 13087, 2, {'a', 'e'}, 26174)]
[('be', 16998, 2, {'e', 'b'}, 33996)]
[('ce', 49875, 2, {'c', 'e'}, 99750)]
[('de', 18454, 2, {'d', 'e'}, 36908)]
[('e', 18455, 1, {'e'}, 18455)]


['abcde', 'abde', 'bcde', 'abe', 'acde', 'bce']

In [151]:
getBenefit(views,lattice,selectedViews)
selectedViews

[]
[('abce', 4219, 1, {'abce'}, 4219)]
[]
[]
[]
[]
[('ace', 48535, 1, {'ace'}, 48535)]
[('ade', 36309, 3, {'d', 'a', 'e'}, 108927)]
[]
[('bde', 16260, 3, {'d', 'e', 'b'}, 77755)]
[('cde', 49843, 3, {'d', 'c', 'e'}, 149529)]
[('ae', 13087, 2, {'a', 'e'}, 26174)]
[('be', 16998, 2, {'e', 'b'}, 33996)]
[('ce', 31137, 2, {'c', 'e'}, 62274)]
[('de', 18454, 2, {'d', 'e'}, 36908)]
[('e', 18455, 1, {'e'}, 18455)]


['abcde', 'abde', 'bcde', 'abe', 'acde', 'bce', 'cde']

In [152]:
getBenefit(views,lattice,selectedViews)
selectedViews

[]
[('abce', 4219, 1, {'abce'}, 4219)]
[]
[]
[]
[]
[('ace', 48535, 1, {'ace'}, 48535)]
[('ade', 36309, 3, {'d', 'a', 'e'}, 108927)]
[]
[('bde', 16260, 3, {'d', 'e', 'b'}, 48780)]
[]
[('ae', 13087, 2, {'a', 'e'}, 26174)]
[('be', 16998, 2, {'e', 'b'}, 33996)]
[('ce', 32, 2, {'c', 'e'}, 64)]
[('de', 64, 2, {'d', 'e'}, 128)]
[('e', 65, 1, {'e'}, 65)]


['abcde', 'abde', 'bcde', 'abe', 'acde', 'bce', 'cde', 'ade']

In [153]:
getBenefit(views,lattice,selectedViews)
selectedViews

[]
[('abce', 4219, 1, {'abce'}, 4219)]
[]
[]
[]
[]
[('ace', 48535, 1, {'ace'}, 48535)]
[]
[]
[('bde', 16260, 3, {'d', 'e', 'b'}, 101349)]
[]
[('ae', 13087, 2, {'a', 'e'}, 26174)]
[('be', 16998, 2, {'e', 'b'}, 33996)]
[('ce', 32, 2, {'c', 'e'}, 64)]
[('de', 64, 2, {'d', 'e'}, 128)]
[('e', 65, 1, {'e'}, 65)]


['abcde', 'abde', 'bcde', 'abe', 'acde', 'bce', 'cde', 'ade', 'bde']

In [154]:
lattice 

{'abcde': 1023860,
 'abce': 1019641,
 'abde': 18456,
 'acde': 290355,
 'bcde': 49909,
 'abe': 975300,
 'ace': 241820,
 'ade': 54765,
 'bce': 31171,
 'bde': 2196,
 'cde': 66,
 'ae': 31543,
 'be': 1458,
 'ce': 34,
 'de': 2,
 'e': 1}

### Trinidad Serna Improved Hypercube views materialization

In [80]:
#Lattice views with only one dimension. Notation (storageCost,computationCost). 
#These values are specified by the user.
latticeDim1={
    'a':(19,5000), 'b':(5329,18000), 'c':(12,12),'d':(4,5) 
}

#For each view count and input the number of atributes in total for each dimension
atributes_perDimension={
    'a':24, 'b':2, 'c':4,'d':2 
}

#Full lattice with all the views, must use the letters of each dimension, i.e. a,b,c,etc to
#indicate the dependencies, for example acd is the parent view of ac,ad,cd,a,c and d.
lattice={
    'abcd': 53799,
    'abc':47133,'abd':18456,'acd':603,'bcd':32918,
    'ab':13973, 'ac':184, 'ad':58, 'bc':26058,'bd':8186,'cd':48,
    'a':19, 'b':5329, 'c':12,'d':4 
    
}

views=list(lattice.keys())
selectedViews=['abcd']

newLattice=lattice
for view in latticeDim1:
    newLattice[view]=latticeDim1[view]

### Ulises Teyechea Lattice Data

In [211]:
#Lattice views with only one dimension. Notation (storageCost,computationCost). 
#These values are specified by the user.
latticeDim1={
    'ae':(31543,31543), 'be':(1458,2192), 'ce':(34,34),'de':(2,2) 
}

#For each view count and input the number of atributes in total for each dimension
atributes_perDimension={
    'ae':1, 'be':4, 'ce':1,'de':1 
}

#Full lattice with all the views, must use the letters of each dimension, i.e. a,b,c,etc to
#indicate the dependencies, for example acd is the parent view of ac,ad,cd,a,c and d.
lattice={
    'abcde': 1023860,
    'abce':1019641,'abde':18456,'acde':290355,'bcde':49909,
    'abe':975300, 'ace':241820, 'ade':54765, 'bce':31171,'bde':2196,'cde':66,
    'ae':31543, 'be':1458, 'ce':34,'de':2,
    'e':1
}

views=list(lattice.keys())
selectedViews=['abcde']

newLattice=lattice
for view in latticeDim1:
    newLattice[view]=latticeDim1[view]

In [212]:
lattice

{'abcde': 1023860,
 'abce': 1019641,
 'abde': 18456,
 'acde': 290355,
 'bcde': 49909,
 'abe': 975300,
 'ace': 241820,
 'ade': 54765,
 'bce': 31171,
 'bde': 2196,
 'cde': 66,
 'ae': (31543, 31543),
 'be': (1458, 2192),
 'ce': (34, 34),
 'de': (2, 2),
 'e': 1}

In [213]:
#Due to lack of time this algorithm will only work for up
#to views made of 4 dimensions maximum
def computationCost(lattice):
    latticeDim2={}
    latticeDim3={}
    latticeDim4={}
    for view in lattice:
        if len(view)==3: #select views at dim=2 or higher
            dimension=list(view)
            latticeDim2[view]=(lattice[view] , \
                               lattice[ dimension[0]+'e'][1] * lattice[ dimension[1]+'e' ][1]  
                              )
            print(view,latticeDim2[view])
        
    for view in latticeDim2:
        newLattice[view]=latticeDim2[view]

            
    for view in lattice:
        if len(view)==4:
            dimension=list(view)
            view3dim=''.join(dimension[:2])
            latticeDim3[view]=( lattice[view],\
                               lattice[view3dim+'e'][1] +\
                               lattice[view3dim+'e'][0] * lattice[dimension[-2]+'e'][1]
            
            )
            print(view,latticeDim3[view])    
            
    for view in latticeDim3:
        newLattice[view]=latticeDim3[view]

    """
    for view in lattice:
        if len(view)==5:
            dimension=list(view)
            view3dim=''.join(dimension[:2])
            view4dim=''.join(dimension[:3])
            latticeDim4[view]=( lattice[view],\
                               lattice[view3dim][1] +\
                               lattice[view3dim][0] * lattice[dimension[-2]][1] +\
                               lattice[view4dim][0] * lattice[dimension[-2]][1]
                              )
            #print(view,latticeDim4[view])    
"""        
    for view in latticeDim4:
        newLattice[view]=latticeDim4[view]
    
    return newLattice

In [214]:
newLattice

{'abcde': 1023860,
 'abce': 1019641,
 'abde': 18456,
 'acde': 290355,
 'bcde': 49909,
 'abe': 975300,
 'ace': 241820,
 'ade': 54765,
 'bce': 31171,
 'bde': 2196,
 'cde': 66,
 'ae': (31543, 31543),
 'be': (1458, 2192),
 'ce': (34, 34),
 'de': (2, 2),
 'e': 1}

In [215]:
newLattice = computationCost(newLattice)
newLattice

abe (975300, 69142256)
ace (241820, 1072462)
ade (54765, 63086)
bce (31171, 74528)
bde (2196, 4384)
cde (66, 68)
abce (1019641, 102302456)
abde (18456, 71092856)
acde (290355, 1556102)
bcde (49909, 136870)


{'abcde': 1023860,
 'abce': (1019641, 102302456),
 'abde': (18456, 71092856),
 'acde': (290355, 1556102),
 'bcde': (49909, 136870),
 'abe': (975300, 69142256),
 'ace': (241820, 1072462),
 'ade': (54765, 63086),
 'bce': (31171, 74528),
 'bde': (2196, 4384),
 'cde': (66, 68),
 'ae': (31543, 31543),
 'be': (1458, 2192),
 'ce': (34, 34),
 'de': (2, 2),
 'e': 1}

In [216]:
def probabilityOfChange(atributes_perDimension,lattice,accepted_probability_of_db_change_in_literature):
    totalElements={}
    probability={}
    for view in lattice:
        countElements=0
        dimensions=list(view)
        for dimension in dimensions:
            if dimension in atributes_perDimension: #Consistency check for notation
                countElements+=atributes_perDimension[dimension]
        
        totalElements[view]=countElements+len(view)
        #print(view,totalElements[view])
        
    max_totalElements_key=max(totalElements, key=totalElements.get)
    for view in lattice:
        probability[view]= totalElements[view] * \
        accepted_probability_of_db_change_in_literature / \
        totalElements[max_totalElements_key]
        
    return probability

In [217]:
probability=probabilityOfChange(atributes_perDimension,newLattice,0.2)
probability

{'abcde': 0.2,
 'abce': 0.16,
 'abde': 0.16,
 'acde': 0.16,
 'bcde': 0.16,
 'abe': 0.12000000000000002,
 'ace': 0.12000000000000002,
 'ade': 0.12000000000000002,
 'bce': 0.12000000000000002,
 'bde': 0.12000000000000002,
 'cde': 0.12000000000000002,
 'ae': 0.08,
 'be': 0.08,
 'ce': 0.08,
 'de': 0.08,
 'e': 0.04}

In [219]:
lattice={
    'abcde': 1023860,
    'abce':1019641,'abde':18456,'acde':290355,'bcde':49909,
    'abe':975300, 'ace':241820, 'ade':54765, 'bce':31171,'bde':2196,'cde':66,
    'ae':31543, 'be':1458, 'ce':34,'de':2,
    'e':1
}

In [220]:
lattice

{'abcde': 1023860,
 'abce': 1019641,
 'abde': 18456,
 'acde': 290355,
 'bcde': 49909,
 'abe': 975300,
 'ace': 241820,
 'ade': 54765,
 'bce': 31171,
 'bde': 2196,
 'cde': 66,
 'ae': 31543,
 'be': 1458,
 'ce': 34,
 'de': 2,
 'e': 1}

In [221]:
def getGreedyBenefit(views,lattice,selectedViews):
    B={}
    for view in lattice:
        storageCostDifference=0
        benefit=[]
        dependentViews=[]
        if view not in selectedViews:
            parent=getParentView(view,lattice,selectedViews)       
            storageCostDifference=abs(lattice[view]-lattice[parent])
            
            #Count the number of dependent Views
            view_dependentViews=dependencyCheck(view,views)     
            
            materialized_view_dependentViews=[]
            
            for selectedView in selectedViews:
                if selectedView != parent and selectedView!=selectedViews[0]:
                    materialized_view_dependentViews.extend(dependencyCheck(selectedView,views))
                
            dependentViews=\
            set(view_dependentViews).difference(set(materialized_view_dependentViews))
            if dependentViews==set():
                dependentViews=set(view)
            #estimate the difference only for those selected nodes that are not parents
        
            addBenefit=additionalBenefit(views,lattice)
            if addBenefit and addBenefit[1]== view:  
                benefit.append((view,storageCostDifference,len(dependentViews),\
                dependentViews,storageCostDifference*len(dependentViews)+addBenefit[0]))
                B[view]=storageCostDifference*len(dependentViews)+addBenefit[0]
            else:
                benefit.append((view,storageCostDifference,len(dependentViews),\
                dependentViews,storageCostDifference*len(dependentViews)))
                B[view]=storageCostDifference*len(dependentViews)
                
        #print(benefit)

    #Select the view with the highest benefit    
    #update=max(B.items(), key=operator.itemgetter(1))[0]
    #selectedViews.append(update)
        
    return B

In [222]:
views=list(lattice.keys())
selectedViews=['abcde']
greedyBenefit=getGreedyBenefit(views,lattice,selectedViews)
greedyBenefit

{'abce': 33752,
 'abde': 8043232,
 'acde': 5868040,
 'bcde': 7791608,
 'abe': 194240,
 'ace': 3128160,
 'ade': 3876380,
 'bce': 3970756,
 'bde': 4086656,
 'cde': 4095176,
 'ae': 1984634,
 'be': 2044804,
 'ce': 2047652,
 'de': 2047716,
 'e': 1023859}

In [227]:
def getTriniBenefit(views,selectedViews,greedyBenefit,newLattice,probability):
    newBenefit={}
    fq={}
    for view in views:
        if view not in selectedViews and view != 'e':
            fq[view]=len(dependencyCheck(view,views))
            newBenefit[view]=\
            greedyBenefit[view] * fq[view] - \
            (\
            (newLattice[view][1] / fq[view] ) *\
            (1.0 + probability[view]) 
            )
            
            print (view,greedyBenefit[view] * fq[view], \
                   newLattice[view][1] / fq[view],\
                   (1 + probability[view]), newBenefit[view])
    
    #Select the view with the highest benefit    
    update=max(newBenefit.items(), key=operator.itemgetter(1))[0]
    selectedViews.append(update)
    
    print(selectedViews)
    return selectedViews

In [228]:
selectedViews=['abcde']
getTriniBenefit(views,selectedViews,greedyBenefit,newLattice,probability)

abce 270016 12787807.0 1.16 -14563840.12
abde 64345856 8886607.0 1.16 54037391.88
acde 46944320 194512.75 1.16 46718685.21
bcde 62332864 17108.75 1.16 62313017.85
abe 776960 17285564.0 1.12 -18582871.680000003
ace 12512640 268115.5 1.12 12212350.64
ade 15505520 15771.5 1.12 15487855.92
bce 15883024 18632.0 1.12 15862156.16
bde 16346624 1096.0 1.12 16345396.48
cde 16380704 17.0 1.12 16380684.96
ae 3969268 15771.5 1.08 3952234.78
be 4089608 1096.0 1.08 4088424.32
ce 4095304 17.0 1.08 4095285.64
de 4095432 1.0 1.08 4095430.92
['abcde', 'bcde']


['abcde', 'bcde']

In [229]:
greedyBenefit=getGreedyBenefit(views,lattice,selectedViews)
getTriniBenefit(views,selectedViews,greedyBenefit,newLattice,probability)

abce 135008 12787807.0 1.16 -14698848.12
abde 32172928 8886607.0 1.16 21864463.880000003
acde 23472160 194512.75 1.16 23246525.21
abe 388480 17285564.0 1.12 -18971351.680000003
ace 6256320 268115.5 1.12 5956030.64
ade 7752760 15771.5 1.12 7735095.92
bce 299808 18632.0 1.12 278940.16
bde 763408 1096.0 1.12 762180.48
cde 797488 17.0 1.12 797468.96
ae 2021366 15771.5 1.08 2004332.78
be 193804 1096.0 1.08 192620.32
ce 199500 17.0 1.08 199481.64
de 199628 1.0 1.08 199626.92
['abcde', 'bcde', 'acde']


['abcde', 'bcde', 'acde']

In [230]:
greedyBenefit=getGreedyBenefit(views,lattice,selectedViews)
getTriniBenefit(views,selectedViews,greedyBenefit,newLattice,probability)

abce 67504 12787807.0 1.16 -14766352.12
abde 16086464 8886607.0 1.16 5777999.880000001
abe 194240 17285564.0 1.12 -19165591.680000003
ace 388280 268115.5 1.12 87990.63999999996
ade 1884720 15771.5 1.12 1867055.92
bce 1186640 18632.0 1.12 1165772.16
bde 381704 1096.0 1.12 380476.48
cde 598116 17.0 1.12 598096.96
ae 517624 15771.5 1.08 500590.78
be 96902 1096.0 1.08 95718.32
ce 199500 17.0 1.08 199481.64
de 199628 1.0 1.08 199626.92
['abcde', 'bcde', 'acde', 'abde']


['abcde', 'bcde', 'acde', 'abde']

In [231]:
greedyBenefit=getGreedyBenefit(views,lattice,selectedViews)
getTriniBenefit(views,selectedViews,greedyBenefit,newLattice,probability)

abce 33752 12787807.0 1.16 -14800104.12
abe 3827376 17285564.0 1.12 -15532455.680000003
ace 194140 268115.5 1.12 -106149.36000000004
ade 435708 15771.5 1.12 418043.92
bce 74952 18632.0 1.12 54084.159999999996
bde 195120 1096.0 1.12 193892.48
cde 671676 17.0 1.12 671656.96
ae 52348 15771.5 1.08 35314.78
be 67992 1096.0 1.08 66808.32
ce 199500 17.0 1.08 199481.64
de 73816 1.0 1.08 73814.92
['abcde', 'bcde', 'acde', 'abde', 'cde']


['abcde', 'bcde', 'acde', 'abde', 'cde']

In [232]:
greedyBenefit=getGreedyBenefit(views,lattice,selectedViews)
getTriniBenefit(views,selectedViews,greedyBenefit,newLattice,probability)

abce 33752 12787807.0 1.16 -14800104.12
abe 3827376 17285564.0 1.12 -15532455.680000003
ace 194140 268115.5 1.12 -106149.36000000004
ade 435708 15771.5 1.12 418043.92
bce 74952 18632.0 1.12 54084.159999999996
bde 195120 1096.0 1.12 193892.48
ae 52348 15771.5 1.08 35314.78
be 67992 1096.0 1.08 66808.32
ce 128 17.0 1.08 109.64
de 256 1.0 1.08 254.92
['abcde', 'bcde', 'acde', 'abde', 'cde', 'ade']


['abcde', 'bcde', 'acde', 'abde', 'cde', 'ade']

In [233]:
greedyBenefit=getGreedyBenefit(views,lattice,selectedViews)
getTriniBenefit(views,selectedViews,greedyBenefit,newLattice,probability)

abce 33752 12787807.0 1.16 -14800104.12
abe 3827376 17285564.0 1.12 -15532455.680000003
ace 194140 268115.5 1.12 -106149.36000000004
bce 169328 18632.0 1.12 148460.16
bde 195120 1096.0 1.12 193892.48
ae 52348 15771.5 1.08 35314.78
be 67992 1096.0 1.08 66808.32
ce 128 17.0 1.08 109.64
de 256 1.0 1.08 254.92
['abcde', 'bcde', 'acde', 'abde', 'cde', 'ade', 'bde']


['abcde', 'bcde', 'acde', 'abde', 'cde', 'ade', 'bde']

In [234]:
greedyBenefit=getGreedyBenefit(views,lattice,selectedViews)
getTriniBenefit(views,selectedViews,greedyBenefit,newLattice,probability)

abce 33752 12787807.0 1.16 -14800104.12
abe 3827376 17285564.0 1.12 -15532455.680000003
ace 194140 268115.5 1.12 -106149.36000000004
bce 74952 18632.0 1.12 54084.159999999996
ae 52348 15771.5 1.08 35314.78
be 2952 1096.0 1.08 1768.32
ce 4452 17.0 1.08 4433.64
de 256 1.0 1.08 254.92
['abcde', 'bcde', 'acde', 'abde', 'cde', 'ade', 'bde', 'bce']


['abcde', 'bcde', 'acde', 'abde', 'cde', 'ade', 'bde', 'bce']

In [235]:
greedyBenefit=getGreedyBenefit(views,lattice,selectedViews)
getTriniBenefit(views,selectedViews,greedyBenefit,newLattice,probability)

abce 33752 12787807.0 1.16 -14800104.12
abe 3827376 17285564.0 1.12 -15532455.680000003
ace 194140 268115.5 1.12 -106149.36000000004
ae 52348 15771.5 1.08 35314.78
be 2952 1096.0 1.08 1768.32
ce 128 17.0 1.08 109.64
de 62594 1.0 1.08 62592.92
['abcde', 'bcde', 'acde', 'abde', 'cde', 'ade', 'bde', 'bce', 'de']


['abcde', 'bcde', 'acde', 'abde', 'cde', 'ade', 'bde', 'bce', 'de']

In [236]:
greedyBenefit=getGreedyBenefit(views,lattice,selectedViews)
getTriniBenefit(views,selectedViews,greedyBenefit,newLattice,probability)

abce 33752 12787807.0 1.16 -14800104.12
abe 3827376 17285564.0 1.12 -15532455.680000003
ace 194140 268115.5 1.12 -106149.36000000004
ae 52348 15771.5 1.08 35314.78
be 2952 1096.0 1.08 1768.32
ce 128 17.0 1.08 109.64
['abcde', 'bcde', 'acde', 'abde', 'cde', 'ade', 'bde', 'bce', 'de', 'ae']


['abcde', 'bcde', 'acde', 'abde', 'cde', 'ade', 'bde', 'bce', 'de', 'ae']

In [237]:
greedyBenefit=getGreedyBenefit(views,lattice,selectedViews)
getTriniBenefit(views,selectedViews,greedyBenefit,newLattice,probability)

abce 33752 12787807.0 1.16 -14800104.12
abe 3827376 17285564.0 1.12 -15532455.680000003
ace 194140 268115.5 1.12 -106149.36000000004
be 63122 1096.0 1.08 61938.32
ce 128 17.0 1.08 109.64
['abcde', 'bcde', 'acde', 'abde', 'cde', 'ade', 'bde', 'bce', 'de', 'ae', 'be']


['abcde', 'bcde', 'acde', 'abde', 'cde', 'ade', 'bde', 'bce', 'de', 'ae', 'be']